# <span style="color: blue;">Декораторы</span>

### Синтаксис использования декораторов

**Декоратор** — функция, которая принимает другую функцию и возвращает функцию.


In [ ]:
@trace
def foo(x):
    return 42

Аналогичная по смыслу версия без синтаксического сахара:

In [ ]:
def foo(x):
    return 42

foo = trace(foo)

Теперь понятно, что по имени `foo` будет доступно то, что вернула функция `trace`. 

Это и есть результат применения декоратора.

Возвращаемый объект может быть любого типа.

## Теория декораторов

### Пример: @trace

Декоратор `trace` выводит на экран сообщение с информацией о вызове декорируемой функции.

In [ ]:
def trace(func):
     def inner(*args, **kwargs):
         print(func.__name__, args, kwargs)
         return func(*args, **kwargs)
     return inner

Применим его к тождественной функции:

In [ ]:
@trace
def identity(x):
    "I do nothing useful."
    return x

identity(42)

### Пример: что дальше?

**1).** Проблема с `help` и атрибутами декорируемой функции.

In [ ]:
help(identity)                                             

**2).** Хотелсь бы возможность глобально отключать `trace` без лишних телодвижений.

**3).** Явное указание файла при использовании `trace` (параметры для декоратора)


In [ ]:
@trace(sys.stderr)
def identity(x):
    return x

**4).** Использование `sys.stdout` для вывода по умолчанию.

### Декораторы и help: проблема

In [ ]:
def identity(x):
    "I do nothing useful."
    return x

In [ ]:
identity.__name__, identity.__doc__

In [ ]:
identity = trace(identity)
identity.__name__, identity.__doc__

### `__module__`

У любой функции есть атрибут `__module__`, содержащий имя модуля, в котором функция была определена.

Для функций, определённых в интерпретаторе, например:

In [ ]:
identity.__module__

### Декораторы и help: решение "в лоб"

Давайте просто возьмём и установим “правильные”
значения в атрибуты декорируемой функции:

In [ ]:
def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    inner.__module__ = func.__module__
    inner.__name__ = func.__name__
    inner.__doc__ = func.__doc__
    return inner

Проверим:

In [ ]:
@trace
def identity(x):
    "I do nothing useful."
    return x

identity.__name__, identity.__doc__

### Декораторы и help: модуль functools

В модуле `functools` из стандартной библиотеки Python есть функция, реализующая логику копирования внутренних атрибутов


In [ ]:
import functools

def trace(func):
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    functools.update_wrapper(inner, func)
    return inner

То же самое можно сделать с помощью декоратора `wraps`:

In [ ]:
def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner

### Управление поведением trace

Заведём глобальную переменную `trace_enabled` и с её помощью будем отключать и включать trace.

In [ ]:
trace_enabled = False

def trace(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner if trace_enabled else func

Если `trace` выключен, то результатом применения декоратора является сама функция `func` — никаких
дополнительных действий в момент её исполнения производиться не будет.

Происходит это в момент компиляции в байт-код

In [ ]:
trace_enabled = False

def f(): 
    pass

f = trace(f)  # функция `f` уже без изменений

trace_enabled = True  # декоратор уже применён не будет

### Декораторы с аргументами: синтаксис

Напоминание:

In [ ]:
@trace
def identity(x):
    return x

In [ ]:
def identity(x):
    return x

identity = trace(identity)

Для декораторов с аргументами эквивалентность
сохраняется

In [ ]:
import sys

In [ ]:
@trace(sys.stderr)
def identity(x):
    return x

In [ ]:
def identity(x):
    return x

deco = trace(sys.stderr)  # здесь возвращается декоратор
identity = deco(identity)  # применяем декоратор к функции

### Декораторы с аргументами: реализация

In [ ]:
def trace(handle):
    def decorator(func):  # описание декоратора, который будет возвращаться
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs,
                file=handle)
            return func(*args, **kwargs)
        return inner
    return decorator  # здесь возвращается декоратор

### Декораторы с опциональными аргументами: магическая версия

In [ ]:
def trace(func=None, *, handle=sys.stdout):
    # со скобками
    if func is None:
        return lambda f: trace(f, handle=handle)

    # без скобок
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return inner

In [ ]:
@trace
def f():
    pass

# f = trace(f)

In [ ]:
@trace(handle=sys.stderr)
def f():
    pass

# deco = trace(handle=sys.stderr)

**Вопрос:** Зачем требовать, чтобы аргументы декоратора были только ключевыми?

In [ ]:
@trace(sys.stderr)
def f():
    pass

### “Теория” декораторов: резюме

**Декоратор** — способ модифицировать поведение функции, сохраняя читаемость кода.

Декораторы бывают:
* без аргументов `@trace`
* с аргументами `@trace(sys.stderr)`
* с опциональными аргументами

## Практика декораторов

### Пример: @timethis

In [ ]:
import functools
import time

def timethis(func=None, *, n_iter=100):
    # n_iter -- сколько раз нужно вызвать функцию
    
    if func is None:
        return lambda func: timethis(func, n_iter=n_iter)

    @functools.wraps(func)
    def inner(*args, **kwargs):
        print(func.__name__, end=" ... ")
        acc = float("inf")
        for i in range(n_iter):
            tick = time.perf_counter()
            result = func(*args, **kwargs)
            acc = min(acc, time.perf_counter() - tick)
        print(acc)
        return result
    return inner

result = timethis(sum)(range(10 ** 6))

### Пример: @profiled

In [ ]:
def profiled(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        inner.ncalls += 1
        return func(*args, **kwargs)

    inner.ncalls = 0
    return inner

@profiled
def identity(x):
    return x

In [ ]:
identity(42)

In [ ]:
identity.ncalls

Через атрибут прокидываем данные наружу

### Пример: @once

In [ ]:
def once(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        if not inner.called:
            func(*args, **kwargs)
            inner.called = True
    inner.called = False
    return inner

@once
def initialize_settings():
    print("Settings initialized.")

In [ ]:
initialize_settings()

In [ ]:
initialize_settings()

Можно было бы использовать `nonlocal` вместо атрибута

**Вопрос**: Как модифицировать декоратор `@once`, чтобы он поддерживал
функции, возвращающие не `None` значения?

### Пример: @memoized

**Мемоизация** — сохранение результатов выполнения функции для предотвращения избыточных вычислений.

Напишем декоратор для автоматической мемоизации “любой” функции.

Полезно, например, для решения задач динамического программирования.

In [ ]:
def memoized(func):
    cache = {}  # локальная переменная

    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args, kwargs
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    
    return inner

### Пример: @memoized и функция Аккермана

In [ ]:
@memoized
def ackermann(m, n):
    if not m:
        return n + 1
    elif not n:
        # в момент вызова мы ищем и находим мемоизированную версию `ackermann`
        return ackermann(m - 1, 1)  
    else:
        return ackermann(m - 1, ackermann(m, n - 1))

ackermann(3, 4)

**Вопрос**: Что же делать?

### Пример: снова @memoized

Частное решение проблемы:

In [ ]:
def memoized(func):
    cache = {}

    @functools.wraps(func)
    def inner(*args, **kwargs):
        key = args + tuple(sorted(kwargs.items()))
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return inner

ackermann(3, 4)

**Вопрос:** Кстати, почему частное?

Нет универсального и быстрого решения. 

Можно сериализовывать аргументы в строку, например, через `str` или, что более осмысленно, через `pickle`.

### Пример: @deprecated

In [13]:
import warnings

def deprecated(func):
    code = func.__code__
    warnings.warn_explicit(
        func.__name__ + " is deprecated.",
        category=DeprecationWarning,
        filename=code.co_filename,
        lineno=code.co_firstlineno + 1
    )
    return func

@deprecated
def identity(x):
    return x

Возможно лучше было бы делать через обёртку, тогда `warning` был бы во время вызова функции

### Контракты pre и post

**Контрактное программирование** — способ проектирования программ, основывающийся на формальном описании интерфейсов в терминах предусловий, постусловий и инвариантов.

В Python контрактное программирование можно реализовать в виде библиотеки декораторов:<br/>
https://pypi.python.org/pypi/contracts

In [ ]:
@pre(lambda x: x >= 0, "negative argument")
def checked_log(x):
    pass

is_not_nan = post(lambda r: not math.isnan(r),
                  "not a number")
@is_not_nan
def something_useful():
    pass

### Реализация @pre

In [ ]:
import math

def pre(cond, message):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            assert cond(*args, **kwargs), message
            return func(*args, **kwargs)
        return inner
    return wrapper

@pre(lambda x: x >= 0, "negative argument")
def checked_log(x):
    return math.log(x)

checked_log(-42)

### Реализация @post

In [ ]:
def post(cond, message):
    def wrapper(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            result = func(*args, **kwargs)
            assert cond(result), message
            return result
        return inner
    return wrapper

@post(lambda x: not math.isnan(x), "not a number")
def something_useful():
    return float("nan")

something_useful()

### Цепочки декораторов

Синтаксис Python разрешает одновременное применение
нескольких декораторов.

Порядок декораторов имеет значение:

In [ ]:
def square_arg(func):
    return lambda x: func(x * x)

def add_to_arg(func):
    return lambda x: func(x + 42)

@square_arg
@add_to_arg
def identity(x):
    return x

identity(2)

In [ ]:
@add_to_arg
@square_arg
def identity(x):
    return x

identity(2)

Чтобы было более понятно:

In [ ]:
identity = square(addsome(identity))

Т.е. вначале двойку возведём в квадрат и передадим дальше.

### Использование декораторов: резюме

Декораторы в мире Python вездесущи и полезны.


Больше примеров можно найти по следующей ссылке и практически в любой библиотеке на Python.<br/>
https://wiki.python.org/moin/PythonDecoratorLibrary

## Декораторы и классы

Синтаксис декораторов работает не только для функций, но и для классов.

In [ ]:
@deco
class Noop:
    pass

In [ ]:
class Noop:
    pass

Noop = deco(Noop)

В этом случае декоратор -- это функция, которая принимает класс и возвращает другой, возможно, преобразованный, класс.

Декораторы классов можно также использовать вместо классов-примесей.

Например, `ThreadSafeMixin` из прошлой лекции:

In [2]:
def thread_safe(cls):
    # запоминаем ссылки на оригинальные методы
    orig_increment = cls.increment
    orig_get = cls.get

    # конструируем две функции, которые заменят эти методы:
    def increment(self):
        with self.get_lock():
            orig_increment(self)

    def get(self):
        with self.get_lock():
            return orig_get(self)

    cls.get_lock = ...  # добавляем атрибут
    cls.increment = increment  # заменяем методы
    cls.get = get
    return cls  # возвращаем новый класс

### Декоратор класса: @singleton

In [8]:
import functools

def singleton(cls):
    instance = None

    @functools.wraps(cls)  # все атрибуты класса скопируются в функцию-обёртку
    def inner(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance

    return inner  # подменим класс на функцию-обёртку

@singleton
class Noop:
    "I do nothing at all."

In [5]:
id(Noop())

2985508716

In [6]:
id(Noop())

2985508716

In [20]:
Noop() is Noop()

True

### Декоратор класса: @deprecated

In [15]:
import warnings

def deprecated(cls):
    orig_init = cls.__init__  # запоминаем оригинальный __init__

    @functools.wraps(cls.__init__)  # декорируем __init__
    def new_init(self, *args, **kwargs):
        warnings.warn(
            cls.__name__ + " is deprecated.",
            category=DeprecationWarning
        )
        orig_init(self, *args, **kwargs)
            
    cls.__init__ = new_init  # подменяем __init__
    return cls

@deprecated
class Counter:
    def __init__(self, initial=0):
        self.value = initial

In [16]:
c = Counter()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Counter is deprecated.
  # Remove the CWD from sys.path while we load stuff.


## Декоратор в виде класса

In [19]:
import functools
import sys

class trace:
    def __init__(self, handle):
        self.handle = handle

    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(func.__name__, args, kwargs,
                  file=self.handle)
            return func(*args, **kwargs)
        return inner

@trace(sys.stderr)  # обязательно принимает аргументы
def identity(x):
    return x

identity(42)

identity (42,) {}


42

Как бы разделили две фазы

## Модуль functools

### functools.lru_cache

Родственник уже рассмотренного `memoized`, сохраняющий *фиксированное* количество последних вызовов.

Познакомим `lru_cache` с функцией Аккермана

In [ ]:
@functools.lru_cache(maxsize=64)
def ackermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackermann(m - 1, 1)  
    else:
        return ackermann(m - 1, ackermann(m, n - 1))

ackermann(3, 4)

In [ ]:
ackermann.cache_info()

По умолчанию `maxsize=128`

Можно не ограничивать количество сохраняемых вызовов, тогда мы получим в точности поведение `memoized`:

In [ ]:
@functools.lru_cache(maxsize=None)
def ackermann(m, n):
    if not m:
        return n + 1
    elif not n:
        return ackermann(m - 1, 1)  
    else:
        return ackermann(m - 1, ackermann(m, n - 1))

**Вопрос**: Почему использовать `None` в качестве значения по умолчанию для `maxsize` — плохая идея?

### functools.partial

С помощью `partial` можно зафиксировать часть позиционных и ключевых аргументов в функции.

Пример:

In [ ]:
sort_by_values = functools.partial(sorted, key=lambda p: p[1])
sort_by_values([("a", 4), ("b", 2)])
# sort_by_values(d.items())

In [ ]:
g = functools.partial(sorted, [2, 3, 1, 4])
g()

In [ ]:
g(key=lambda x: -x)

### Обобщённые функции

Функция `len` называется **обобщённой**, так как её реализация может быть специализирована для конкретного типа.

In [ ]:
len([1, 2, 3, 4])

In [ ]:
len({1, 2, 3, 4})

In [ ]:
len(range(4))

Примеры других обобщённых функций в Python:

In [ ]:
str([1, 2, 3, 4])

In [ ]:
hash((1, 2, 3, 4))

In [ ]:
sum([[1], [2]], [])

Подробнее: http://python.org/dev/peps/pep-0443

Как же реализовать свою обобщённую функцию?

### `functools.singledispatch`

В качестве примера реализуем функцию `pack`, которая сериализует объект в компактное строковое представление

In [ ]:
@functools.singledispatch
def pack(obj):  # базовый случай (если никакой другой не подошёл)
    type_name = type(obj).__name__
    assert False, "Unsupported type: " + type_name

Научим функцию `pack` сериализовывать числа и списки (конкретные типы)

In [ ]:
@pack.register(int)
def _(obj):
    return b"I" + hex(obj).encode("ascii")

@pack.register(list)
def _(obj):
    return b"L" + b",".join(map(pack, obj))

In [ ]:
pack([1, 2, 3])

In [ ]:
pack(42.)

### Мотивация для reduce

Свёртка.

Рассмотрим пример:

In [ ]:
sum([1, 2, 3, 4], start=0)

In [ ]:
(((0 + 1) + 2) + 3) + 4

А что, если мы хотим использовать другую бинарную
операцию, например, умножение?


In [ ]:
((1 * 2) * 3) * 4

Функция `reduce` обобщает логику функции `sum` на произвольную бинарную операцию.

In [ ]:
functools.reduce(lambda acc, x: acc * x, [1, 2, 3, 4])

Есть ещё интересные частные случаи:
* `any(...)`
* `all(...)`

### Подробнее о reduce

Функция `reduce` принимает три аргумента: бинарную функцию, последовательность и опциональное начальное значение.

Вычисление `reduce(op, xs, initial)` можно схематично представить как:

Несколько примеров:

In [ ]:
functools.reduce(lambda acc, d: 10 * acc + int(d),
                 "1914", initial=0)

functools.reduce(merge, [[1, 2, 7], [5, 6], [0]])

### reduce и философия

Несмотря на свою популярность в функциональных языках, в Python довольно сложно придумать полезный пример использования `reduce`.

Резюме про `reduce`:
* работает с любым объектом, поддерживающим протокол итератора;
* работает слева направо;
* использует первый элемент последовательности, если начальное значение не указано явно.

### Модуль functools: резюме

Модуль `functools` украшает будни любителя функционального программирования.

Мы поговорили про:
* `lru_cache`
* `partial`
* `singledispatch`
* `reduce`